In [1]:
import os, sys

sys.path.append('../Pose_Estimation')

# Keras/TF dependencies
import keras
from keras.models import load_model, Sequential, Model
from keras.layers import Input, Dense, Activation
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
import tensorflow as tf
from keras import backend as K

# Common deps
import numpy as np
import cv2
import util
from config_reader import config_reader

Using TensorFlow backend.


In [2]:
param_, model_ = config_reader()
PYTORCH_WEIGHTS_PATH = model_['pytorch_model']
TENSORFLOW_WEIGHTS_PATH = model_['tensorflow_model'].replace('./', '../Pose_Estimation/')
USE_MODEL = model_['use_model']
USE_GPU = param_['use_gpu']
TORCH_CUDA = lambda x: x.cuda() if USE_GPU else x

In [3]:
class TensorFlowModel:
    """
    TensorFlow model credited to Michal F.
    (https://github.com/michalfaber/keras_Realtime_Multi-Person_Pose_Estimation)
    """

    def __init__(self, load_weights=True, compressed_model=None):
        self.session = tf.Session()
        # K.set_session(self.session)

        def relu(x):
            return Activation('relu')(x)

        def conv(x, nf, ks, name):
            x1 = Conv2D(nf, (ks, ks), padding='same', name=name)(x)
            return x1

        def pooling(x, ks, st, name):
            x = MaxPooling2D((ks, ks), strides=(st, st), name=name)(x)
            return x

        def vgg_block(x):

            # Block 1
            x = conv(x, 64, 3, "conv1_1")
            x = relu(x)
            x = conv(x, 64, 3, "conv1_2")
            x = relu(x)
            x = pooling(x, 2, 2, "pool1_1")

            # Block 2
            x = conv(x, 128, 3, "conv2_1")
            x = relu(x)
            x = conv(x, 128, 3, "conv2_2")
            x = relu(x)
            x = pooling(x, 2, 2, "pool2_1")

            # Block 3
            x = conv(x, 256, 3, "conv3_1")
            x = relu(x)
            x = conv(x, 256, 3, "conv3_2")
            x = relu(x)
            x = conv(x, 256, 3, "conv3_3")
            x = relu(x)
            x = conv(x, 256, 3, "conv3_4")
            x = relu(x)
            x = pooling(x, 2, 2, "pool3_1")

            # Block 4
            x = conv(x, 512, 3, "conv4_1")
            x = relu(x)
            x = conv(x, 512, 3, "conv4_2")
            x = relu(x)

            # Additional non vgg layers
            x = conv(x, 256, 3, "conv4_3_CPM")
            x = relu(x)
            x = conv(x, 128, 3, "conv4_4_CPM")
            x = relu(x)

            return x

        def stage1_block(x, num_p, branch):

            # Block 1
            x = conv(x, 128, 3, "conv5_1_CPM_L%d" % branch)
            x = relu(x)
            x = conv(x, 128, 3, "conv5_2_CPM_L%d" % branch)
            x = relu(x)
            x = conv(x, 128, 3, "conv5_3_CPM_L%d" % branch)
            x = relu(x)
            x = conv(x, 512, 1, "conv5_4_CPM_L%d" % branch)
            x = relu(x)
            x = conv(x, num_p, 1, "conv5_5_CPM_L%d" % branch)

            return x

        def stageT_block(x, num_p, stage, branch, prefix='Heatmap'):

            # Block 1
            with tf.name_scope('%sBlock' % (prefix)):
                x = conv(x, 128, 7, "Mconv1_stage%d_L%d" % (stage, branch))
                x = relu(x)
                x = conv(x, 128, 7, "Mconv2_stage%d_L%d" % (stage, branch))
                x = relu(x)
                x = conv(x, 128, 7, "Mconv3_stage%d_L%d" % (stage, branch))
                x = relu(x)
                x = conv(x, 128, 7, "Mconv4_stage%d_L%d" % (stage, branch))
                x = relu(x)
                x = conv(x, 128, 7, "Mconv5_stage%d_L%d" % (stage, branch))
                x = relu(x)
                x = conv(x, 128, 1, "Mconv6_stage%d_L%d" % (stage, branch))
                x = relu(x)
                x = conv(x, num_p, 1, "Mconv7_stage%d_L%d" % (stage, branch))

            return x

        # Hyper-parameters
        input_shape = (None,None,3)
        img_input = Input(shape=input_shape)

        stages = 6
        np_branch1 = 38
        np_branch2 = 19

        # output resize operations
        # TODO: probably better of batch resizing at the end instead of doing them discretely
        self.raw_heatmap = tf.placeholder(tf.float32, shape=(None, None, None, 19))
        self.raw_paf = tf.placeholder(tf.float32, shape=(None, None, None, 38))
        self.resize_size = tf.placeholder(tf.int32, shape=(2))

        self.resize_heatmap = tf.transpose(tf.image.resize_images(self.raw_heatmap, self.resize_size, align_corners=True), perm=[0, 3, 1, 2])
        self.resize_paf = tf.transpose(tf.image.resize_images(self.raw_paf, self.resize_size, align_corners=True), perm=[0, 3, 1, 2])

        if compressed_model:
            print '| Loading compressed model:', compressed_model
            self.model = load_model(compressed_model)

            compressed_weights = compressed_model.replace('.h5', '_w.h5')
            self.model.load_weights(compressed_weights)
            return

        # VGG
        with tf.name_scope('VggConvLayer'):
            stage0_out = vgg_block(img_input)

        # stage 1
        with tf.name_scope('DualLayer%d' % (1)):
            stage1_branch1_out = stage1_block(stage0_out, np_branch1, 1)
            stage1_branch2_out = stage1_block(stage0_out, np_branch2, 2)
            x = Concatenate()([stage1_branch1_out, stage1_branch2_out, stage0_out])

        # stage t >= 2
        for sn in range(2, stages + 1):
            with tf.name_scope('DualLayer%d' % (sn)):
                stageT_branch1_out = stageT_block(x, np_branch1, sn, 1, prefix='Heat')
                stageT_branch2_out = stageT_block(x, np_branch2, sn, 2, prefix='PAF')
                if (sn < stages):
                    x = Concatenate()([stageT_branch1_out, stageT_branch2_out, stage0_out])

        self.model = Model(img_input, [stageT_branch1_out, stageT_branch2_out])
        if load_weights:
            self.model.load_weights(TENSORFLOW_WEIGHTS_PATH)

    def evaluate(self, oriImg, scale=1.0):
        imageToTest = cv2.resize(oriImg, (0,0), fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
        imageToTest_padded, pad = util.padRightDownCorner(imageToTest, model_['stride'], model_['padValue'])
        input_img = np.transpose(np.float32(imageToTest_padded[:,:,:,np.newaxis]), (3,0,1,2))/256 - 0.5;

        output1, output2 = self.model.predict(input_img)

        # Replicating bilinear upsampling to heatmaps procedure.
        resize_dict = {
            self.resize_size: [oriImg.shape[0], oriImg.shape[1]],
            self.raw_heatmap: output2,
            self.raw_paf: output1,
        }

        heatmap, paf = self.session.run([self.resize_heatmap, self.resize_paf], feed_dict=resize_dict)
        heatmap, paf = heatmap[0], paf[0]

        return (output1, output2), (heatmap, paf)

In [4]:
model = TensorFlowModel()

In [7]:
import matplotlib.pyplot as plt
import numpy.linalg as la

show_plots = False

print 'Generating filter rankings...'

filter_rankings = {}
for layer_ii, layer in enumerate(model.model.layers):
    if layer.get_weights():
        wmat, bmat = layer.get_weights()
        dist = np.zeros(wmat.shape[3])
        for dim_j in range(wmat.shape[3]):
            ijsum = 0
            for dim_i in range(wmat.shape[2]):
                Fij = wmat[:, :, dim_i, dim_j]
                onenorm = la.norm(Fij, ord=1)
                ijsum += onenorm
            dist[dim_j] = ijsum
        sort_inds = np.argsort(dist)
        filter_rankings[layer.name] = sort_inds

        relmax = np.max(dist - np.min(dist))
        sorted_vals = (dist[sort_inds] - np.min(dist)) / relmax 

        if layer_ii % 10 is 0: 
            print '\rLayer %d/%d' % (layer_ii + 1, len(model.model.layers))
        if show_plots:
            plt.title(layer.name)
            plt.scatter(range(len(dist)), sorted_vals)

            plt.show()
        

    

Generating filter rankings...
Layer 2/181
Layer 4/181
Layer 7/181
Layer 9/181
Layer 12/181
Layer 14/181
Layer 16/181
Layer 18/181
Layer 21/181
Layer 23/181
Layer 25/181
Layer 27/181
Layer 29/181
Layer 30/181
Layer 33/181
Layer 34/181
Layer 37/181
Layer 38/181
Layer 41/181
Layer 42/181
Layer 45/181
Layer 46/181
Layer 48/181
Layer 49/181
Layer 52/181
Layer 53/181
Layer 56/181
Layer 57/181
Layer 60/181
Layer 61/181
Layer 64/181
Layer 65/181
Layer 68/181
Layer 69/181
Layer 72/181
Layer 73/181
Layer 75/181
Layer 76/181
Layer 79/181
Layer 80/181
Layer 83/181
Layer 84/181
Layer 87/181
Layer 88/181
Layer 91/181
Layer 92/181
Layer 95/181
Layer 96/181
Layer 99/181
Layer 100/181
Layer 102/181
Layer 103/181
Layer 106/181
Layer 107/181
Layer 110/181
Layer 111/181
Layer 114/181
Layer 115/181
Layer 118/181
Layer 119/181
Layer 122/181
Layer 123/181
Layer 126/181
Layer 127/181
Layer 129/181
Layer 130/181
Layer 133/181
Layer 134/181
Layer 137/181
Layer 138/181
Layer 141/181
Layer 142/181
Layer 145/181
L